<a href="https://colab.research.google.com/github/bewithankit/CS3DP19/blob/main/PySpark_vs_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
from pyspark import SparkContext, SparkConf
import time

# Initialize Spark
conf = SparkConf().setAppName('wordCount')
sc = SparkContext(conf=conf)

# Start timing
start_time = time.time()

# Read the text file and perform word count
linesRDD = sc.textFile("/content/ulysses.txt")
wordsRDD = linesRDD.flatMap(lambda line: line.split())
wordCounts = wordsRDD.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Collect the results
word_counts_collected = wordCounts.collect()

# Stop timing
end_time = time.time()

# Print the word counts
print("\nWord count:\n", word_counts_collected)

# Print the time taken
print("Time taken with PySpark: {} seconds".format(end_time - start_time))

# Stop Spark
sc.stop()

In [ ]:
from dask.distributed import Client
import dask.bag as db
import time

# Initialize Dask Client
client = Client()

# Start timing
start_time = time.time()

# Read the text file into a Dask Bag and perform word count
lines = db.read_text("/content/ulysses.txt")
word_counts = lines.map(str.split).flatten().frequencies(sort=True)

# Compute the results
word_counts_computed = word_counts.compute()

# Stop timing
end_time = time.time()

# Print the word counts
print(word_counts_computed)

# Print the time taken
print("Time taken with Dask: {} seconds".format(end_time - start_time))

# Shut down the Dask client
client.shutdown()
